In [1]:
!pip install photutils

In [2]:
# The standard fare:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.visualization import ZScaleInterval
%matplotlib inline
# Recall our use of this module to work with FITS files in Lab 4:
from astropy.io import fits 

# This lets us use various Unix (or Unix-like) commands within Python:
import os 

# We will see what this does shortly.
import glob 
import astropy.time

import astropy.stats as stat
from astropy.stats import mad_std
from astropy.stats import sigma_clip
from photutils.utils import calc_total_error
import astropy.stats as stat

from photutils import aperture_photometry, CircularAperture, CircularAnnulus, DAOStarFinder

In [3]:
#changing directory into where shift_methods is
os.chdir('C:/Users/batma/Documents/reduction_combination_pipeline')

In [4]:
import shift_methods as p
from importlib import reload
%reload_ext autoreload 
%autoreload 2 
reload(p)

<module 'shift_methods' from 'C:\\Users\\batma\\Documents\\reduction_combination_pipeline\\shift_methods.py'>

In [5]:
def filesorter(filename, foldername, fitskeyword_to_check, keyword):
    '''
    Takes .fit files with a specific keyword value in their header and sends them to the specified
    folder-- if the folder does't exist, it creates it.
    
    Inputs:
    filename (str) - name of .fits file
    foldername (str) - name of folder that the .fits file may be sorted to
    fitskeyword_to_check (str) - if the value of the header category keyword matches this input, then the .fits file will 
                                    be sorted to foldername
    keyword (str) - the header category that is being checked
    '''
    # Checking to see if the filename exists, if it doesn't, it prints that the filename does not exist or that it has been moved.  
    if os.path.exists(filename):
        pass
    else:
        print(filename + " does not exist or has already been moved.")
        return
    #acquiring all 'column' headers for specified filename, assigning it to header
    #then assigning values under specified header keyword to fits_type
    header = fits.getheader(filename)
    fits_type = header[keyword]
    
    # Checking to see if foldername exists. If it does, move on, if it doesn't, it generates a folder of that name.
    if os.path.exists(foldername):
        pass
    else:
        print("Making new directory: " + foldername)
        os.mkdir(foldername)
    
    # Checks to see if the fits_type we got from the keyword header matches the fitskeyword we are looking for. If it does, it moves it into the destination which is specified by the
    # foldername we inputted into the function. If it does not, it just doesn't do anything. 
    if fits_type == fitskeyword_to_check:
        destination = foldername + '/'
        print("Moving " + filename + " to: ./" + destination + filename)
        os.rename(filename, destination + filename)  
    return

In [6]:
def sortall(master_directory):
    '''
    Loops through each directory in a given master directory for darks and flats, sorts them based on EXPTIME and FILTER.
    Inputs:
    master_directory (str) - the directory where all of the raw data folders are
    '''
    for root, dirs, files in os.walk(master_directory):
        for i in dirs:
            if 'Darks' in i:
                os.chdir('C:/Users/batma/Documents/reduction_combination_pipeline'+"/"+i)
                dark_fits = glob.glob('*.fit')
                times = []
                for i in dark_fits:
                    headers = fits.getheader(i)
                    times.append(headers['EXPTIME'])
                times = np.array(times)
                uniquetimes = np.unique(times)
                for i in range(len(uniquetimes)):
                    for yeet in dark_fits:
                        filesorter(yeet, str(uniquetimes[i]) + 'sec', uniquetimes[i], 'EXPTIME')
            elif 'Flats' in i:
                os.chdir('C:/Users/batma/Documents/reduction_combination_pipeline'+"/"+i)
                all_fits = glob.glob('*.fit')
                for filename in all_fits:
                    filesorter(filename, 'Blue', 'B', 'FILTER')
                    filesorter(filename, 'Visual', 'V', 'FILTER')
                    filesorter(filename, 'Red', 'R', 'FILTER')  
    return

In [7]:
master_dir = 'C:/Users/batma/Documents/reduction_combination_pipeline'
sortall(master_dir)

Making new directory: 40.0sec
Moving T11-smithcollege-Dark-040-LD20210203-LT100809-BIN2-001.fit to: ./40.0sec/T11-smithcollege-Dark-040-LD20210203-LT100809-BIN2-001.fit
Moving T11-smithcollege-Dark-040-LD20210203-LT100857-BIN2-002.fit to: ./40.0sec/T11-smithcollege-Dark-040-LD20210203-LT100857-BIN2-002.fit
Moving T11-smithcollege-Dark-040-LD20210203-LT100946-BIN2-003.fit to: ./40.0sec/T11-smithcollege-Dark-040-LD20210203-LT100946-BIN2-003.fit
Moving T11-smithcollege-Dark-040-LD20210203-LT101035-BIN2-004.fit to: ./40.0sec/T11-smithcollege-Dark-040-LD20210203-LT101035-BIN2-004.fit
Moving T11-smithcollege-Dark-040-LD20210203-LT101126-BIN2-005.fit to: ./40.0sec/T11-smithcollege-Dark-040-LD20210203-LT101126-BIN2-005.fit
Moving T11-smithcollege-Dark-040-LD20210203-LT101215-BIN2-006.fit to: ./40.0sec/T11-smithcollege-Dark-040-LD20210203-LT101215-BIN2-006.fit
Moving T11-smithcollege-Dark-040-LD20210203-LT101304-BIN2-007.fit to: ./40.0sec/T11-smithcollege-Dark-040-LD20210203-LT101304-BIN2-007.f

In [2]:
def mediancombinenan(filelist):
    '''
    The function takes 1 input, a list of fits files to be combined, and takes the median
    pixel values for each pixel, outputting a median fits file. This function ignores naN values.
    
    Inputs:
    filelist (array-like) - a list of strings that are filepaths to images to be combined
    '''
    # Defines variable holding the length of the inputted list of files.
    n = len(filelist)
    
    # Assigns a name to the array data of the first file in the list.
    first_frame_data = fits.getdata(filelist[0])
    
        # Puts the y and x dimensions of the image into variables.
    imsize_y, imsize_x = first_frame_data.shape
    
    # Creates an empty three-dimensional array which will store the data from the entire list of files.
    fits_stack = np.zeros((imsize_y, imsize_x , n)) 
    
    # Goes through the list of files and sets the corresponding element in the array to the value of the
    # current pixel.
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        hed = fits.getheader(filelist[ii])
        if (hed['EXPTIME']> 60):
            im_normal = im/hed['EXPTIME']
            im_final = im_normal*60
        else: im_final = im
        fits_stack[:,:,ii] = im_final
        
    # Uses a numpy function to create a median frame along the third axis, which it then returns.        
    med_frame = np.nanmedian(fits_stack, axis = 2)
    
    return med_frame

def mediancombinescience(filelist):
    '''
    The function takes 1 input, a list of fits files to be combined, and takes the median
    pixel values for each pixel, outputting a median fits file.
    
    Inputs:
    filelist (array-like) - a list of strings that are filepaths to images to be combined
    '''
    # Defines variable holding the length of the inputted list of files.
    n = len(filelist)
    
    # Assigns a name to the array data of the first file in the list.
    first_frame_data = fits.getdata(filelist[0])
    
    # Puts the y and x dimensions of the image into variables.
    imsize_y, imsize_x = first_frame_data.shape
    
    # Creates an empty three-dimensional array which will store the data from the entire list of files.
    fits_stack = np.zeros((imsize_y, imsize_x , n)) 
    
    # Goes through the list of files and sets the corresponding element in the array to the value of the
    # current pixel.
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        fits_stack[:,:,ii] = im
        
    # Uses a numpy function to create a median frame along the third axis, which it then returns.        
    med_frame = np.nanmedian(fits_stack, axis = 2)
    
    return med_frame

def bias_subtract(filename, path_to_bias):
    '''
    This function takes two inputs: name of a file and a file path to the master bias and subtracts the master bias from the data
    It creates a file with the prefix 'b_', the new data, the header, and overwrites any existing files with the same name
    
    Inputs:
    filename (str) - filepath to image that is to be bias subtracted
    path_to_bias (str) - filepath to master bias image
    '''
    #grab information from input filepaths
    data = fits.getdata(filename)
    header = fits.getheader(filename)
    mb = fits.getdata(path_to_bias)
    
    #bias subtraction
    subtracted = data - mb

    #save the file
    fits.writeto('b_' + filename, subtracted, header, overwrite=True)
    return 

def getmasterbias(biasdirectory):
    '''
    This function creates a master bias frame given the path to the bias frames.
    
    Inputs:
    biasdirectory (str) - filepath to the biasframes folder
    
    Outputs:
    master_bias_path (str) - filepath to the master bias image
    '''
    
    os.chdir(biasdirectory)
    biasfiles = glob.glob('*Bias*.fit')
    #median combine all bias frames, and save the master
    fits.writeto('Master_Bias.fit', mediancombinescience(biasfiles), fits.getheader(biasfiles[0]), overwrite=True)
    master_bias_path = os.getcwd() + '/Master_Bias.fit'
    return(master_bias_path)
    
def getmasterdark(master_bias_path, darkdirectory):
    '''
    This function creates a master dark frame given the path to the dark frames.
    
    Inputs:
    master_bias_path (str) - filepath to the master bias image
    darkdirectory (str) - filepath to the darks folder
    
    Outputs:
    master_darkpath (str) - filepath to the master dark image
    '''
    
    os.chdir(darkdirectory)
    darkfiles = glob.glob('*.fit')
    #first must bias subtract all darks
    for i in darkfiles:
        bias_subtract(i, master_bias_path)
    biassubtracteddarks = glob.glob('b_*.fit')
    #combine to create a master dark
    CBSD = mediancombinescience(biassubtracteddarks)
    #save master dark
    fits.writeto('Master_Dark.fit', CBSD, fits.getheader(biassubtracteddarks[0]), overwrite = True)
    master_darkpath = os.getcwd() + '/Master_Dark.fit'
    return(master_darkpath)

def dark_subtract(filename, path_to_dark):
    '''
    Takes an image file name and a path to the master dark frame,
    normalizes the dark current and scales it to the image exposure time,
    subtracts it, and writes the resulting file to the working directory.
    
    Inputs:
    filename (str) - filepath an image
    path_to_dark (str) - filepath to the master dark image
    '''
    # Your code goes here.
    
    fileframe = fits.getdata(filename)
    fileheader = fits.getheader(filename)
    darkframe = fits.getdata(path_to_dark)
    darkheader = fits.getheader(path_to_dark)
    if (darkheader['EXPTIME'] != fileheader['EXPTIME']):
        masterdarknormalized = darkframe/darkheader['EXPTIME']
        exposure = fileheader['EXPTIME']
        darktouse = masterdarknormalized*exposure
    else:
        darktouse = darkframe

    newfile = fileframe - darktouse

    prefix = 'd'
    
    fits.writeto(prefix + filename, newfile, fileheader, overwrite=True) # finish this code too to save the FITS. Make sure it has the correct header!
    return

            
def flatfield_correction(filepath, pathtomastervisualflatfield, pathtomasterblueflatfield, pathtomasterredflatfield):
    '''
    Applies a flatfield correction to a images within a folder
    
    Inputs:
    filepath (str) - filepath to folder
    pathtomastervisualflatfield (str) - filepath to master V flat
    pathtomasterblueflatfield (str) - filepath to master B flat
    pathtomasterredflatfield (str) - filepath to master R flat
    '''
    os.chdir(filepath)
    reducedfits = glob.glob('db_*.fit')
    #loop through images 
    for i in reducedfits:
        #check image filter
        header = fits.getheader(i)
        print(header['FILTER'])
        if header['FILTER'] == 'V':
            path_master_flatfield = pathtomastervisualflatfield
        elif header['FILTER'] == 'B':
            path_master_flatfield = pathtomasterblueflatfield
        elif header['FILTER'] == 'R':
            path_master_flatfield = pathtomasterredflatfield
        
        #do the flatfield correction
        flatdata = fits.getdata(path_master_flatfield)
        data = fits.getdata(i)
        output = data/flatdata
        yeet = np.where(np.isnan(output))
        output[yeet] = 0
        #save the flat fielded image
        prefix = 'f'
        fits.writeto(prefix + i, output, header, overwrite=True)
    ff_corrected = glob.glob('fdb_*.fit')
    #sort the fully reduced images by filter
    for fitsfile in ff_corrected:
        filesorter(fitsfile, 'ReducedV', 'V', 'FILTER')
        filesorter(fitsfile, 'ReducedB', 'B', 'FILTER')
        filesorter(fitsfile, 'ReducedR', 'R', 'FILTER')        
    

def process_images(path_to_science, master_bias_path, master_dark_path, pathtomastervisualflatfield, pathtomasterblueflatfield, pathtomasterredflatfield):
    '''
    Performs bias, dark, and flat field corrections on all science images, then sorts them by FILTER.
    
    Inputs:
    path_to_science (str) - filepath to the lights folder
    master_bias_path (str) - filepath to master bias
    master_dark_path (str) - filepath to master dark
    pathtomastervisualflatfield (str) - filepath to master V flat
    pathtomasterblueflatfield (str) - filepath to master B flat
    pathtomasterredflatfield (str) - filepath to master R flat
    '''
    os.chdir(path_to_science)
    fitsz = glob.glob('*.fit')
    print(fitsz)
    for i in fitsz:
        bias_subtract(i, master_bias_path)
    biassubtracted = glob.glob('b_*.fit')
    for i in biassubtracted:
        dark_subtract(i, master_dark_path)
    flatfield_correction(path_to_science, pathtomastervisualflatfield,pathtomasterblueflatfield, pathtomasterredflatfield)


In [10]:
path_to_master_bias = getmasterbias('C:/Users/batma/Documents/reduction_combination_pipeline/Bias')

In [11]:
master_dark_path_60sec = getmasterdark(path_to_master_bias, 'C:/Users/batma/Documents/reduction_combination_pipeline/Darks/60.0sec')
master_dark_path_100sec = getmasterdark(path_to_master_bias, 'C:/Users/batma/Documents/reduction_combination_pipeline/Darks/100.0sec')
master_dark_path_40sec = getmasterdark(path_to_master_bias, 'C:/Users/batma/Documents/reduction_combination_pipeline/Darks/40.0sec')

In [12]:
path_to_master_B_flatfield = 'C:/Users/batma/Documents/reduction_combination_pipeline/Flats/Blue/Copy of Master_Flat B 1_B_2004x1336_Bin2x2_Temp-25C_ExpTime49s.fit'
path_to_master_R_flatfield = 'C:/Users/batma/Documents/reduction_combination_pipeline/Flats/Red/Copy of Master_Flat R 2_R_2004x1336_Bin2x2_Temp-25C_ExpTime17s.fit'
path_to_master_V_flatfield = 'C:/Users/batma/Documents/reduction_combination_pipeline/Flats/Visual/Copy of Master_Flat V 2_V_2004x1336_Bin2x2_Temp-25C_ExpTime20s.fit'

In [13]:
def process_images_large(master_directory, path_to_master_bia, master_dark_path_60se,path_to_master_V_flatfiel,path_to_master_B_flatfiel, path_to_master_R_flatfiel):
    '''
    Applies process_images function to each night of data inside master directory
    '''
    for root, dirs, files in os.walk(master_directory):
            for i in dirs:
                if '2021' in i:
                    science_path_1 = 'C:/Users/batma/Documents/reduction_combination_pipeline'+"/"+i
                    process_images(science_path_1, path_to_master_bia, master_dark_path_60se,path_to_master_V_flatfiel,path_to_master_B_flatfiel, path_to_master_R_flatfiel  )

In [14]:
process_images_large(master_dir, path_to_master_bias, master_dark_path_60sec,path_to_master_V_flatfield,path_to_master_B_flatfield, path_to_master_R_flatfield  )

['raw-T11-smithcollege-NGC2264-20210302-223235-B-BIN2-W-100-001.fit', 'raw-T11-smithcollege-NGC2264-20210302-223507-B-BIN2-W-100-002.fit', 'raw-T11-smithcollege-NGC2264-20210302-223719-B-BIN2-W-100-003.fit', 'raw-T11-smithcollege-NGC2264-20210302-223930-B-BIN2-W-100-004.fit', 'raw-T11-smithcollege-NGC2264-20210302-224141-B-BIN2-W-100-005.fit', 'raw-T11-smithcollege-NGC2264-20210302-224353-B-BIN2-W-100-006.fit', 'raw-T11-smithcollege-NGC2264-20210302-224604-V-BIN2-W-060-001.fit', 'raw-T11-smithcollege-NGC2264-20210302-224742-V-BIN2-W-060-002.fit', 'raw-T11-smithcollege-NGC2264-20210302-224905-V-BIN2-W-060-003.fit', 'raw-T11-smithcollege-NGC2264-20210302-225031-V-BIN2-W-060-004.fit', 'raw-T11-smithcollege-NGC2264-20210302-225156-V-BIN2-W-060-005.fit', 'raw-T11-smithcollege-NGC2264-20210302-225320-V-BIN2-W-060-006.fit', 'raw-T11-smithcollege-NGC2264-20210302-225445-R-BIN2-W-040-001.fit', 'raw-T11-smithcollege-NGC2264-20210302-225604-R-BIN2-W-040-002.fit', 'raw-T11-smithcollege-NGC2264-202

Moving fdb_raw-T11-smithcollege-Cone Nebula-20210307-191401-B-BIN2-E-100-004.fit to: ./ReducedB/fdb_raw-T11-smithcollege-Cone Nebula-20210307-191401-B-BIN2-E-100-004.fit
fdb_raw-T11-smithcollege-Cone Nebula-20210307-191401-B-BIN2-E-100-004.fit does not exist or has already been moved.
Moving fdb_raw-T11-smithcollege-Cone Nebula-20210307-191624-B-BIN2-E-100-005.fit to: ./ReducedB/fdb_raw-T11-smithcollege-Cone Nebula-20210307-191624-B-BIN2-E-100-005.fit
fdb_raw-T11-smithcollege-Cone Nebula-20210307-191624-B-BIN2-E-100-005.fit does not exist or has already been moved.
Moving fdb_raw-T11-smithcollege-Cone Nebula-20210307-191841-B-BIN2-E-100-006.fit to: ./ReducedB/fdb_raw-T11-smithcollege-Cone Nebula-20210307-191841-B-BIN2-E-100-006.fit
fdb_raw-T11-smithcollege-Cone Nebula-20210307-191841-B-BIN2-E-100-006.fit does not exist or has already been moved.
Moving fdb_raw-T11-smithcollege-Cone Nebula-20210307-192055-V-BIN2-E-060-001.fit to: ./ReducedV/fdb_raw-T11-smithcollege-Cone Nebula-20210307-

In [15]:
def alignandcombineBband(master_directory):
    '''
    Aligns and Combines each night of science data in the B-band to create master images for each of them. 
    '''
    for root, dirs, files in os.walk(master_directory):
            for i in dirs:
                if '2021' in i:
                    science_path_B = master_dir +"/"+i+"/ReducedB"
                    os.chdir(science_path_B)
                    lightslist = glob.glob('fdb_*.fit')
                    xshift = []
                    yshift = []
                    for i in lightslist:
                        reference_image = lightslist[0]
                        if 'BIN2-W' in reference_image:
                            initial_state = 'BIN2-W'
                        elif 'BIN2-E' in reference_image:
                            initial_state = 'BIN2-E'
                        if initial_state in i:
                            x_temp, y_temp = p.cross_image(fits.getdata(reference_image), fits.getdata(i), 1000, 500, 800)
                            xshift.append(x_temp)
                            yshift.append(y_temp)
                        else:
                            flipped = fits.getdata(i)[::-1,::-1]
                            x_temp, y_temp = p.cross_image(fits.getdata(reference_image),flipped, 1000, 500, 800)
                            xshift.append(x_temp)
                            yshift.append(y_temp)
                    reference_image = lightslist[0]
                    for i in np.arange(len(lightslist)):
                        if 'BIN2-W' in reference_image:
                            initial_state = 'BIN2-W'
                        elif 'BIN2-E' in reference_image:
                            initial_state = 'BIN2-E'
                        if initial_state in lightslist[i]:
                            header = fits.getheader(lightslist[i])
                            fits.writeto('shifted' +initial_state + lightslist[i], p.shift_image(fits.getdata(lightslist[i]), xshift[i], yshift[i]), header, overwrite=True)
                        else:
                            header = fits.getheader(lightslist[i])
                            fits.writeto('shifted'+initial_state + lightslist[i], p.shift_image(fits.getdata(lightslist[i])[::-1,::-1], xshift[i], yshift[i]), header, overwrite=True)
                    shiftedlist = glob.glob('shifted*.fit')
                    fits.writeto('MasterScience_B.fit', mediancombinescience(shiftedlist), header, overwrite=True)


In [16]:
alignandcombineBband(master_dir)